# Pulling Beer Company Followers Data with Tweepy
**By:** _Mike Scheibel_ 

In [1]:
import datetime
import tweepy
from pprint import pprint

# I've put my API keys in a .py file called API_keys.py
from Mike_API_Keys import api_key, api_key_secret, access_token, access_token_secret

In [2]:
# Authenticate the Tweepy API
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## Collecting Followers

In [3]:
# I'm putting the handles in a list to iterate through below
handles = ['AmstelLight','Heineken_US', 'LabattUSA', 'yuenglingbeer','StellaArtois','MichelobULTRA', 
           'ModeloUSA', 'GuinnessGB', 'DosEquis', 'CoorsLight', 'budlight', 'naturallight', 
           'MillerLite', 'PabstBlueRibbon']

# Create dictionary to hold follower IDs
id_dict = dict() 

# Intialize dictionary.
for handle in handles :
    id_dict[handle] = []

# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in id_dict :
    
    # Let's grab the first page of IDs
    for page in tweepy.Cursor(api.followers_ids,
                              wait_on_rate_limit=True,
                              wait_on_rate_limit_notify=True, 
                              compression=True,
                              screen_name=handle).pages():

        # The page variable comes back as a list, so we have to use .extend rather than .append
        id_dict[handle].extend(page)
        
        break
        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print(elapsed_time)

0:00:07.682221


In [4]:
# let's make sure it worked
for handle in id_dict :
    print(f"{handle} has {len(id_dict[handle])} followers in our dictionary.")

AmstelLight has 2280 followers in our dictionary.
Heineken_US has 5000 followers in our dictionary.
LabattUSA has 5000 followers in our dictionary.
yuenglingbeer has 5000 followers in our dictionary.
StellaArtois has 5000 followers in our dictionary.
MichelobULTRA has 5000 followers in our dictionary.
ModeloUSA has 5000 followers in our dictionary.
GuinnessGB has 5000 followers in our dictionary.
DosEquis has 5000 followers in our dictionary.
CoorsLight has 5000 followers in our dictionary.
budlight has 5000 followers in our dictionary.
naturallight has 5000 followers in our dictionary.
MillerLite has 5000 followers in our dictionary.
PabstBlueRibbon has 5000 followers in our dictionary.


## Collecting Followers Information

In [5]:
# The code inside this function comes from Brenden Connors
# This will quickly grab information about each follower.
def get_screen_names(twitter_api, list_of_ids, list_for_screen_names):
    
    start=0 #we have feed the API 100 ID's at a time, this will iterate through them
    end=0
    followers=[]
    while end-1 <= len(list_of_ids): #quit when we get past the end of our list
        end += 100 #update the end of our slice
        if end <= len(list_of_ids): #split into if else statement to slice correctly
            try:
                followers_temp = twitter_api.lookup_users(list_of_ids[start:end])
            except tweepy.TweepError as err:
                if err.api_code in (103,88) :
                    print('sleeping, 900 seconds')
                    time.sleep(900)
                else :
                    print(err)
        else:        
            try:
                followers_temp = twitter_api.lookup_users(list_of_ids[start:])
            except tweepy.TweepError as err:
                if err.api_code in (103,88) :
                    print('sleeping, 900 seconds')
                    time.sleep(900)
                else :
                    print(err)
                    
        followers.extend(followers_temp)
        start = end #update our starting slice index for next loop
        
    list_for_screen_names.extend(followers)
        

In [6]:
user_dict = dict() 

for handle in id_dict :
    user_dict[handle] = []
    
for handle in user_dict :
    get_screen_names(api,id_dict[handle],user_dict[handle])

[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]
[{'code': 17, 'message': 'No user matches for specified terms.'}]


In [7]:
for handle in user_dict :
    print(f"{handle} has {len(user_dict[handle])} followers in our dictionary.") 

AmstelLight has 2280 followers in our dictionary.
Heineken_US has 5100 followers in our dictionary.
LabattUSA has 5100 followers in our dictionary.
yuenglingbeer has 5100 followers in our dictionary.
StellaArtois has 5100 followers in our dictionary.
MichelobULTRA has 5100 followers in our dictionary.
ModeloUSA has 5100 followers in our dictionary.
GuinnessGB has 5100 followers in our dictionary.
DosEquis has 5100 followers in our dictionary.
CoorsLight has 5100 followers in our dictionary.
budlight has 5100 followers in our dictionary.
naturallight has 5100 followers in our dictionary.
MillerLite has 5100 followers in our dictionary.
PabstBlueRibbon has 5100 followers in our dictionary.


In [ ]:
headers = ['screen_name','name','id','location','followers_count','friends_count','description']

for team in id_dict.keys():
    
    # Descriptions with emoji or non-Roman letters can cause trouble. Encoding your .txt file in utf-8 will help
    with open(f'{team}_followers.txt','w', encoding='utf-8') as out_file:
        out_file.write('\t'.join(headers) + '\n')

        for idx, ids in enumerate(id_dict[team]):
            
            # For accounts set to private, we won't be able to get the description unless we follow them
            # Putting in a try/except statement, we can get around this issue.
            try:
                user = api.get_user(ids)
                description = str(user.description).replace('\t',' ').replace('\n',' ')
                outline = [user.screen_name,user.name,user.id,
                    user.location,user.followers_count,user.friends_count,
                    user.description]
                
                out_file.write('\t'.join([str(item) for item in outline]) + '\n')
                
            except:
                continue

            